# Modelo de Predição da Copa do Mundo 2022

In [55]:
#importando bibliotecas
import scipy.stats as stats
from scipy.stats import t, norm, probplot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, ceil
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from sklearn import *

%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split

## Base de Dados 1 - Overall Jogadores 2022

### Vizualização e limpeza inicial da base de dados

In [56]:
jogadores2022 = pd.read_excel('players_22.xlsx')
pd.set_option('display.max_columns', None)
jogadorescopa = pd.read_excel('World Cup players.xlsx')

In [57]:
jogadores2022.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel AndrÃ©s Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,72,73.0,Paris Saint-Germain,French Ligue 1,1.0,RW,30.0,NaN,2021-08-10,2023.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,144300000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,81,21.0,FC Bayern MÃ¼nchen,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,1353.0,RS,9.0,Right,4,4,5,High/Medium,Unique,Yes,197200000.0,"#Aerial Threat, #Distance Shooter, #Clinical F...","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,NaN,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,83,11.0,Manchester United,English Premier League,1.0,ST,7.0,NaN,2021-08-27,2023.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,83300000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,NaN,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.pn

In [58]:
jogadores2022.loc[0,:]

sofifa_id                                                      158023
player_url          https://sofifa.com/player/158023/lionel-messi/...
short_name                                                   L. Messi
long_name                             Lionel AndrÃ©s Messi Cuccittini
player_positions                                           RW, ST, CF
                                          ...                        
player_face_url     https://cdn.sofifa.net/players/158/023/22_120.png
club_logo_url                  https://cdn.sofifa.net/teams/73/60.png
club_flag_url                     https://cdn.sofifa.net/flags/fr.png
nation_logo_url              https://cdn.sofifa.net/teams/1369/60.png
nation_flag_url                   https://cdn.sofifa.net/flags/ar.png
Name: 0, Length: 110, dtype: object

In [59]:
jogadores2022= jogadores2022.drop('sofifa_id',axis = 1)
jogadores2022= jogadores2022.drop('player_url',axis = 1)
jogadores2022= jogadores2022.drop('player_face_url',axis = 1)
jogadores2022= jogadores2022.drop('club_logo_url',axis = 1)
jogadores2022= jogadores2022.drop('nation_logo_url',axis = 1)
jogadores2022= jogadores2022.drop('nation_flag_url',axis = 1)
jogadores2022= jogadores2022.drop('club_team_id',axis = 1)
jogadores2022= jogadores2022.drop('club_loaned_from',axis = 1)
jogadores2022= jogadores2022.drop('club_joined',axis = 1)
jogadores2022= jogadores2022.drop('nationality_id',axis = 1)
jogadores2022= jogadores2022.drop('nation_team_id',axis = 1)
jogadores2022= jogadores2022.drop('nation_jersey_number',axis = 1)
jogadores2022= jogadores2022.drop('long_name',axis = 1)


### Escalações e Formato do campeonato

Como a análise será feita com base nos dados individuais das seleções, é necessário primeiramente saber quem são os jogadores que fazem parte de cada uma. Além disso, o esquema do campeonanto a priori é feito em 8 grupos de 4 times cada sorteados arbitrairamente antes da competição. Desses grupos os dois times que tiverem mais vitórias, ou em caso de empate, maior slado de gols passam para as oitavas de final. O chaveamento completo está detalhado abaixo.

![BoxPlot](copaaaaaa.jfif)

### Grupo A - Overall

In [60]:
#Catar
filtro = jogadores2022['nationality_name']=='Qatar'
qatar = jogadores2022.loc[filtro,:]

#Equador
filtro =jogadores2022['nationality_name']=='Ecuador'
ecuador = jogadores2022.loc[filtro,:]

#Senegal
filtro =jogadores2022['nationality_name']=='Senegal'
senegal = jogadores2022.loc[filtro,:]

#Holanda
filtro =jogadores2022['nationality_name']=='Netherlands'
netherlands = jogadores2022.loc[filtro,:]

### Grupo B - Overall

In [61]:
#Inglaterra
filtro = jogadores2022['nationality_name']=='England'
england = jogadores2022.loc[filtro,:]

#Irã
filtro = jogadores2022['nationality_name']=='Iran'
iran = jogadores2022.loc[filtro,:]

#EUA
filtro = jogadores2022['nationality_name']=='USA'
usa = jogadores2022.loc[filtro,:]

#Pais de Gales
filtro = jogadores2022['nationality_name']=='Wales'
wales = jogadores2022.loc[filtro,:]


### Grupo C - Overall

In [62]:
#Argentina
filtro = jogadores2022['nationality_name']=='Argentina'
argentina = jogadores2022.loc[filtro,:]

#Arabia Saudita
filtro = jogadores2022['nationality_name']=='Saudi Arabia'
arabia = jogadores2022.loc[filtro,:]

#Mexico
filtro = jogadores2022['nationality_name']=='Mexico'
mexico = jogadores2022.loc[filtro,:]

#Polonia
filtro = jogadores2022['nationality_name']=='Poland'
poland = jogadores2022.loc[filtro,:]

### Grupo D - Overall

In [63]:
#França
filtro = jogadores2022['nationality_name']=='France'
france = jogadores2022.loc[filtro,:]

#Dinamarca
filtro = jogadores2022['nationality_name']=='Denmark'
denmark = jogadores2022.loc[filtro,:]

#Tunisia
filtro = jogadores2022['nationality_name']=='Tunisia'
tunisia = jogadores2022.loc[filtro,:]

#Australia
filtro = jogadores2022['nationality_name']=='Australia'
australia = jogadores2022.loc[filtro,:]

### Grupo E - Overall

In [64]:
#Espanha
filtro = jogadores2022['nationality_name']=='Spain'
spain = jogadores2022.loc[filtro,:]

#Japao
filtro = jogadores2022['nationality_name']=='Japan'
japan = jogadores2022.loc[filtro,:]

#Costa Rica
filtro = jogadores2022['nationality_name']=='Costa Rica'
costarica = jogadores2022.loc[filtro,:]

#Alemanha
filtro = jogadores2022['nationality_name']=='Germany'
germany = jogadores2022.loc[filtro,:]

### Grupo F - Overall

In [65]:
#Croacia
filtro = jogadores2022['nationality_name']=='Croatia'
croatia = jogadores2022.loc[filtro,:]

#Marrocos
filtro = jogadores2022['nationality_name']=='Morocco'
morocco = jogadores2022.loc[filtro,:]

#Belgica
filtro = jogadores2022['nationality_name']=='Belgium'
belgium = jogadores2022.loc[filtro,:]

#Canada
filtro = jogadores2022['nationality_name']=='Canada'
canada = jogadores2022.loc[filtro,:]


### Grupo G - Overall

In [66]:
#Brasil
filtro = jogadores2022['nationality_name']=='Brazil'
brazil = jogadores2022.loc[filtro,:]

#Suíça
filtro = jogadores2022['nationality_name']=='Switzerland'
switzerland = jogadores2022.loc[filtro,:]

#Camarões
filtro = jogadores2022['nationality_name']=='Cameroon'
cameroon = jogadores2022.loc[filtro,:]

#Sérvia
filtro = jogadores2022['nationality_name']=='Serbia'
serbia = jogadores2022.loc[filtro,:]

### Grupo H - Overall

In [67]:
#Portugal
filtro = jogadores2022['nationality_name']=='Portugal'
portugal = jogadores2022.loc[filtro,:]

#Coreia do Sul
filtro = jogadores2022['nationality_name']=='South Korea'
korea = jogadores2022.loc[filtro,:]

#Uruguai
filtro = jogadores2022['nationality_name']=='Uruguay'
uruguay = jogadores2022.loc[filtro,:]

#Gana
filtro = jogadores2022['nationality_name']=='Ghana'
ghana = jogadores2022.loc[filtro,:]

In [ ]:

#Gana
filtro = jogadores2022['nationality_name']=='Ghana'
ghana = jogadores2022.loc[filtro,:]

## Dataframe de todos jogadores que estão na copa do mundo

In [68]:
jogadorescopa.head()

,National Team,Number,POS,PLAYER NAME,FIRST NAME(S),LAST NAME(S),NAME ON SHIRT,DOB,CLUB,HEIGHT (CM)
0,Argentina,1,Goalkeeper,ARMANI Franco,Franco,ARMANI,ARMANI,1986-10-16,CA River Plate (ARG),189
1,Argentina,2,Defender,FOYTH Juan,Juan Marcos,FOYTH,FOYTH,1998-01-12,Villarreal CF (ESP),188
2,Argentina,3,Defender,TAGLIAFICO Nicolas,NicolÃ¡s Alejandro,TAGLIAFICO,TAGLIAFICO,1992-08-31,Olympique Lyon (FRA),172
3,Argentina,4,Defender,MONTIEL Gonzalo,Gonzalo Ariel,MONTIEL,MONTIEL,1997-01-01,Sevilla FC (ESP),175
4,Argentina,5,Midfielder,PAREDES Leandro,Leandro Daniel,PAREDES,PAREDES,1994-06-29,Juventus FC (ITA),182


In [70]:

jogadorescopa= jogadorescopa.drop('HEIGHT (CM)',axis = 1)
jogadorescopa= jogadorescopa.drop('DOB',axis = 1)

jogadorescopa.head()


,National Team,Number,POS,PLAYER NAME,FIRST NAME(S),LAST NAME(S),NAME ON SHIRT,CLUB
0,Argentina,1,Goalkeeper,ARMANI Franco,Franco,ARMANI,ARMANI,CA River Plate (ARG)
1,Argentina,2,Defender,FOYTH Juan,Juan Marcos,FOYTH,FOYTH,Villarreal CF (ESP)
2,Argentina,3,Defender,TAGLIAFICO Nicolas,NicolÃ¡s Alejandro,TAGLIAFICO,TAGLIAFICO,Olympique Lyon (FRA)
3,Argentina,4,Defender,MONTIEL Gonzalo,Gonzalo Ariel,MONTIEL,MONTIEL,Sevilla FC (ESP)
4,Argentina,5,Midfielder,PAREDES Leandro,Leandro Daniel,PAREDES,PAREDES,Juventus FC (ITA)


### Grupo A - Escalação

In [73]:
#Catar
qatar_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Qatar']

#Equador
ecuador_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Ecuador']

#Senegal
senegal_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Senegal']

#Holanda
netherlands_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Netherlands']

### Grupo B - Escalação

In [ ]:
#Inglaterra
england_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'England']

#Irã
iran_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Iran']

#EUA
usa_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'USA']

#Pais de Gales
wales_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Wales']

### Grupo C - Escalação

In [ ]:
#Argentina
argentina_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Argentina']

#Arabia Saudita
arabia_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Saudi Arabia']

#Mexico
mexico_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Mexico']

#Polonia
poland_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Poland']

### Grupo D - Escalação

In [ ]:
#França
france_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'France']

#Dinamarca
denmark_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Denmark']

#Tunisia
tunisia_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Tunisia']

#Australia
australia_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Australia']

### Grupo E - Escalação

In [ ]:
#Espanha
spain_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Spain']

#Japao
japan_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Japan']

#Costa Rica
costarica_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Costa Rica']

#Alemanha
germany_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Germany']

### Grupo F - Escalação

In [ ]:
#Croacia
croatia_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Croatia']

#Marrocos
morocco_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Morocco']

#Belgica
belgium_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Belgium']

#Canada
canada_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Canada']


### Grupo G - Escalação

In [ ]:
#Brasil
brazil_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Brazil']

#Suíça
switzerland_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Switzerland']

#Camarões
cameroon_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Cameroon']

#Sérvia
serbia_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Serbia']

### Grupo H - Escalação

In [ ]:
#Portugal
portugal_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Portugal']

#Coreia do Sul
korea_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'South Korea']

#Uruguai
uruguay_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Uruguay']

#Gana
ghana_escalacao = jogadorescopa.loc[jogadorescopa['National Team'] == 'Ghana']

#### Adicionando o overall dos jogadores na escalação